##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Gemini 嵌入語料的搜尋重新排序


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/examples/Search_reranking_using_embeddings.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


本記事展示使用 embeddings 重新排列搜尋結果。本教學重點在以下目標：



1.	設定你的開發環境和 API 存取權限，以使用 Gemini。
2.	使用 Gemini 的函式呼叫支援，以存取 Wikipedia API。
3.	透過 Gemini API 對內容進行嵌入。
4.	重新排列搜尋結果。


這是你要如何實施搜尋重新排序：
1. 用戶將進行搜尋查詢。
2. 你將使用維基百科 API 來傳回相關的搜尋結果。
3. 搜尋結果將被嵌進，並透過計算餘弦相似度等距離量度來評估它們的相關性。
4. 最相關的搜尋結果將作為最終答案傳回。


> 此筆記本中的非程式碼資料擁有創用 CC 授權條款 - 依據姓名標示-相同方式分享 CC-BY-SA 4.0，https://creativecommons.org/licenses/by-sa/4.0/legalcode。


## 安裝


In [ ]:
!pip install -q -U google-generativeai

In [ ]:
!pip install -q wikipedia

注意：[`wikipedia` 套件](https://pypi.org/project/wikipedia/) 說明它「設計為易於使用且簡單，不適合進階使用」，且生產或大量使用應改為「使用 [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) 或其他進階的 [Python MediaWiki API 封裝程式](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)」。


In [ ]:
import json
import textwrap

import google.generativeai as genai

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

要執行以下Cell，你的 API 金鑰必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 秘辛中。如果你還沒有 API 金鑰，或者你不確定如何建立 Colab 秘辛，請查看 [身分驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 快速入門指南，以取得範例。


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 定義工具


如前所述，本教學使用 Gemini 的函式呼叫支援來訪問 Wikipedia API。請參閱 [文件](https://ai.google.dev/docs/function_calling) 以了解有關函式呼叫的更多資訊。


### 定義搜尋函式


為了迎合搜尋引擎的需求，你將按照以下方式設計這個功能：


* 針對每個搜尋查詢，搜尋引擎將使用 `wikipedia.search` 方法獲得相關主題。
* 從相關主題中，引擎將選擇 `n_topics(int)` 個候選主題，並將使用 `gemini-pro` 從頁面中擷取相關資訊。
* 引擎將避免重複輸入並維護搜尋記錄。


In [ ]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-pro')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-pro` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = mining_model.generate_content(textwrap.dedent(f"""\
            Extract relevant information
            about user's query: {query}
            From this source:

            {page}

            Note: Do not summarize. Only Extract and return the relevant information
        """))

        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [ ]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Prompt engineering', 'Gemini (language model)']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Prompt engineering"
Information Source: https://en.wikipedia.org/wiki/Prompt_engineering
Fetching page: "Gemini (language model)"
Information Source: https://en.wikipedia.org/wiki/Gemini_(language_model)
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Prompt_engineering
     https://en.wikipedia.org/wiki/Gemini_(language_model)


這是搜尋結果的樣子：


In [ ]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> **Definition:**
> 
> * A large language model (LLM) is an artificial neural network that can perform natural language processing tasks such as text generation, classification, and question answering.
> 
> **Training and Architecture:**
> 
> * LLMs are trained on massive text datasets using self-supervised and semi-supervised learning techniques.
> * They are typically built with transformer-based architectures and can have billions or even trillions of parameters.
> * Prompt engineering is used to optimize the performance of LLMs on specific tasks.
> 
> **Capabilities:**
> 
> * **Text Generation:** LLMs can generate human-like text, including stories, articles, and code.
> * **Language Understanding:** LLMs can understand the meaning and structure of text, including context and sentiment.
> * **Question Answering:** LLMs can answer questions based on their knowledge of the text they have been trained on.
> * **Natural Language Processing:** LLMs can perform various NLP tasks, such as sentiment analysis, machine translation, and named entity recognition.
> 
> **Multimodality:**
> 
> * LLMs can be extended to handle multiple modalities, such as images, audio, and video, by using tokenization methods.
> * This enables them to perform tasks such as image captioning, video summarization, and audio transcription.
> 
> **Properties:**
> 
> * **Scaling Laws:** The performance of LLMs scales with the size of their training data, the number of parameters, and the computational power used for training.
> * **Emergent Abilities:** LLMs exhibit emergent abilities that are not explicitly programmed into them, such as in-context learning and solving complex reasoning problems.
> * **Interpretation:** Researchers are exploring methods to interpret the inner workings of LLMs and understand how they process and generate language.
> 
> **Wider Impact:**
> 
> * LLMs have potential applications in various industries, including healthcare, education, and customer service.
> * They also raise concerns about copyright infringement, algorithmic bias, and the spread of misinformation.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Large_language_model

> **What are LLMs?**
> 
> Large Language Models (LLMs) are a type of generative AI model that can be trained on large amounts of text data to understand and generate human-like language. They are used in a variety of applications, including:
> 
> * **Text generation:** LLMs can generate text that is indistinguishable from human-written text. This can be used for a variety of purposes, such as writing articles, stories, and marketing copy.
> * **Machine translation:** LLMs can be used to translate text from one language to another.
> * **Question answering:** LLMs can be used to answer questions by searching through large amounts of text data.
> * **Summarization:** LLMs can be used to summarize large amounts of text into shorter, more manageable summaries.
> 
> LLMs are still under development, but they have the potential to revolutionize the way we interact with computers. They could make it possible for us to communicate with computers in a more natural way, and they could help us to access and understand information more easily.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Prompt_engineering

> - Large language models (LLMs) are advanced AI models that can process and generate human-like text.
> - LLMs are trained on massive datasets of text and code, which allows them to understand and respond to a wide range of natural language queries.
> - LLMs are used in a variety of applications, including search engines, chatbots, and machine translation.
> - Gemini is a family of LLMs developed by Google DeepMind.
> - Gemini comprises three models: Gemini Ultra, Gemini Pro, and Gemini Nano.
> - Gemini Ultra is designed for "highly complex tasks", Gemini Pro is designed for "a wide range of tasks", and Gemini Nano is designed for "on-device tasks".
> - Gemini was launched on December 6, 2023.
> - Gemini is available through Google Cloud's Vertex AI service.
> - Gemini is trained on and powered by Google's Tensor Processing Units (TPUs).
> - Gemini has outperformed GPT-4, Anthropic's Claude 2, Inflection AI's Inflection-2, Meta's LLaMA 2, and xAI's Grok 1 on a variety of industry benchmarks.
> - Gemini Pro is made available to Google Cloud customers on AI Studio and Vertex AI on December 13, while Gemini Nano will be made available to Android developers as well.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Gemini_(language_model)

### 將工具傳遞給模型


如果你將函式清單傳遞給 `GenerativeModel` 的 `tools` 參數，
它會從函式的簽章和型別提示中擷取架構，然後將架構傳遞給 API 呼叫。作為回應，模型可能會傳回一個 `FunctionCall` 物件，要求呼叫函式。

注意：這種方法只處理 `AllowedTypes = int | float | str | dict | list['AllowedTypes']` 的註解

`GenerativeModel` 將保留對函式本身的參照，以便它稍後 _can_ 在本地執行函式。


In [ ]:
model = genai.GenerativeModel(
    'gemini-pro',
    tools=[wikipedia_search],
    generation_config={'temperature': 0.6})

## 生成支援性的搜尋查詢


為了讓使用者在原本的查詢有更多的支援搜尋查詢，你可以請模型產生更多這樣的查詢。這樣有助於搜尋引擎在全面層級涵蓋提出的問題。


In [ ]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

為了產生富含創意的問題，且種類更隨機，可以將模型溫度參數設定為較高的值。值可以介於包含 [0.0,1.0] 之間。較接近 1.0 的值會產生變化多端且富含創意的回應，而較接近 0.0 的值通常會產生較為直接的回應。


## 啟用自動函式呼叫並呼叫 API


現在開始一個新的聊天，內容為  `enable_automatic_function_calling=True`。啟用後，`genai.ChatSession`  將處理呼叫功能所需的來回步驟，並傳回最終回應：


In [ ]:
model = genai.GenerativeModel(
    'gemini-pro', tools=[wikipedia_search], generation_config={'temperature': 0.6})

chat = model.start_chat(enable_automatic_function_calling=True)

query = "Explain how deep-sea life survives."

res = chat.send_message(instructions.format(query=query))

Searching for "How do deep-sea creatures survive in the deep sea?"
Related search terms: ['Deep-sea fish', 'Deep-sea community', 'Deep sea']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Searching for "What adaptations have deep-sea creatures developed to survive in the deep sea?"
Related search terms: ['Deep-sea fish', 'Deep-sea community', 'Deep sea']
Searching for "What is the pressure at the bottom of the ocean?"
Related search terms: ['DSV Limiting Factor', 'Titan submersible implosion', 'Seabed']
Fetching page: "DSV Limiting Factor"
Information Source: https://en.wikipedia.org/wiki/DSV_Limiting_Factor
Fetching page: "Titan submersible implosion"
Information Source: https://en.wikipedia.org/wiki/Titan_submersible_implosion
Fetching page: "Seabed"

In [ ]:
to_markdown(res.text)

> Deep-sea life survives in the extreme conditions of the deep sea by developing various adaptations, including:
> 
> * **Pressure Resistance:** Deep-sea organisms have evolved to withstand the crushing pressure of the deep sea, which can reach up to 1,000 times the pressure at sea level. Their bodies have a high internal pressure to balance the external pressure and prevent them from being crushed.
> 
> * **Low Light Adaptation:** Many deep-sea fish have evolved large and sensitive eyes to maximize light absorption in the dark depths. Some species have multiple Rh1 opsin genes, which enhance their ability to see in low light conditions.
> 
> * **Bioluminescence:** Many deep-sea creatures produce their own light through bioluminescent organs to attract prey, communicate, or camouflage themselves.
> 
> * **Neutral Buoyancy:** Deep-sea fish often have low tissue density and reduced skeletal weight to maintain neutral buoyancy in the water column. Some species have swim bladders to help them adjust their buoyancy.
> 
> * **Sensing Adaptations:** To navigate and locate prey in the dark, deep-sea creatures rely on highly sensitive senses such as touch, smell, and electroreception.
> 
> * **Slow Metabolism and Energy Conservation:** Deep-sea organisms have adapted to survive on limited food sources. They have slower metabolisms and specialized feeding behaviors, such as lying in wait for prey or using lures to attract it.

檢查額外的引文：


In [ ]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

這樣看起來似乎有效。你可以檢閱對話紀錄，瞭解在函式呼叫中發送和接收的詳細資訊：


In [ ]:
for content in chat.history:
  part = content.parts[0]

  print(f'{content.role} -> ', end='')
  print(json.dumps(type(part).to_dict(part), indent=2))
  print('---' * 20)


user -> {
  "text": "You have access to the Wikipedia API which you will be using\nto answer a user's query. Your job is to generate a list of search queries which\nmight answer a user's question. Be creative by using various key-phrases from\nthe user's query. To generate variety of queries, ask questions which are\nrelated to  the user's query that might help to find the answer. The more\nqueries you generate the better are the odds of you finding the correct answer.\nHere is an example:\n\nuser: Tell me about Cricket World cup 2023 winners.\n\nfunction_call: wikipedia_search(['What is the name of the team that\nwon the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup\n2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What\nwas the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',\n'Who lifted the Cricket World Cup 2023 trophy?'])\n\nThe search function will return a list of article summaries, use these to\nans

在聊天記錄中你可以看到所有 4 個步驟：

1. 使用者傳送查詢。
2. 模型以 `glm.FunctionCall` 回覆，呼叫 `wikipedia_search`，並帶有多個相關搜尋。
3. 由於在建立 `genai.ChatSession` 時你設定 `enable_automatic_function_calling=True`，它執行搜尋功能並傳回文章摘要清單至該模型。
4. 根據提示中的指示，該模型根據這些摘要產生一個最終答案。


##[可選] 手動執行函式呼叫


如果你想要了解幕後發生了什麼事，本節手動執行 <FunctionCall> 以示範。


In [ ]:
import google.ai.generativelanguage as glm # for lower level code

In [ ]:
chat = model.start_chat()

In [ ]:
result = chat.send_message(instructions.format(query=query))

最初模型返回一個 FunctionCall:


In [ ]:
fc = result.candidates[0].content.parts[0].function_call
fc = type(fc).to_dict(fc)
print(json.dumps(fc, indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "How do deep-sea creatures survive the immense pressure of the deep sea?",
      "What adaptations have deep-sea creatures evolved to withstand the extreme conditions of their environment?",
      "How do deep-sea creatures generate their own light?",
      "What is the role of bioluminescence in the deep sea?",
      "How do deep-sea creatures find food in the darkness of the deep sea?",
      "What are the unique challenges faced by deep-sea creatures?",
      "How do deep-sea creatures reproduce in the extreme conditions of their environment?"
    ]
  }
}


In [ ]:
fc['name']

'wikipedia_search'

以生成參數呼叫函式以取得結果。


In [ ]:
summaries = wikipedia_search(**fc['args'])

Searching for "How do deep-sea creatures survive the immense pressure of the deep sea?"
Related search terms: ['Deep-sea community', 'Sea', 'Ocean']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Sea"
Information Source: https://en.wikipedia.org/wiki/Sea
Fetching page: "Ocean"
Information Source: https://en.wikipedia.org/wiki/Ocean
Searching for "What adaptations have deep-sea creatures evolved to withstand the extreme conditions of their environment?"
Related search terms: ['Deep-sea fish', 'Astrobiology', 'Marine life']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Astrobiology"
Information Source: https://en.wikipedia.org/wiki/Astrobiology
Fetching page: "Marine life"
Information Source: https://en.wikipedia.org/wiki/Marine_life
Searching for "How do deep-sea creatures generate their own light?"
Related search terms: ['Bioluminescence', 'The Deep

現在將 `FunctionResult` 傳送至模型。


In [ ]:
response = chat.send_message(
    glm.Content(
      parts=[glm.Part(
          function_response = glm.FunctionResponse(
            name='wikipedia_search',
            response={'result': summaries}
          )
      )]
    )
)

to_markdown(response.text)

> Deep-sea creatures have evolved unique adaptations to survive the immense pressure of the deep sea. These adaptations include having flexible bodies, using hydrostatic pressure to their advantage, and having specialized proteins that protect their cells from damage.
> 
> Deep-sea creatures also have a number of adaptations that help them find food in the darkness of the deep sea. These adaptations include having large eyes that are sensitive to low levels of light, and having long, sensitive tentacles that can detect prey.
> 
> Some deep-sea creatures also have bioluminescent organs that they use to attract prey or communicate with each other.

## 重新排名搜尋結果


嵌入內容的輔助函式：


In [ ]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

有關嵌入式詳細資訊，請參閱[嵌入式指南](https://ai.google.dev/docs/embeddings_guide)。


下一步是定義函式，用於計算兩個嵌入向量的相似度分數。這些分數將有助於你確定哪個嵌入向量與使用者的查詢最相關。


你現在會實作餘弦相似度作為度量。這裡回傳的嵌入向量將為單位長度，因此其 L1 範數 (`np.linalg.norm()`) 將約等於 1。因此，計算餘弦相似度本質上與計算它們的點積分數相同。


In [ ]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### 與使用者查詢相近

現在是時候找出 Wikipedia API 回傳最相關的搜尋結果。

使用 Gemini API 為使用者的查詢和搜尋結果取得內嵌。


In [ ]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

計算相似度分數：


In [ ]:
sim_value = dot_product(search_res, embedded_query)

使用 `np.argmax` 即可選擇最佳候選項。

**使用者輸入：** 說明深海生物如何生存。

**回覆：** 


In [ ]:
print(summaries[np.argmax(sim_value)])

Deep-sea creatures survive the immense pressure of the deep sea by evolving adaptations that allow them to withstand the extreme conditions. These adaptations include having flexible bodies, using hydrostatic pressure to their advantage, and having specialized proteins that protect their cells from damage.

Based on:
  https://en.wikipedia.org/wiki/Sea


### 與假設文件嵌入 (HyDE) 相似性

取自 [Gao et al](https://arxiv.org/abs/2212.10496) 的靈感，此處的目的是使用 `gemini-pro` 的內部知識為使用者的查詢產生範本答案。此假設解答將作為基底以計算所有搜尋結果的相關性。


In [ ]:
hypothetical_ans_model = genai.GenerativeModel('gemini-pro')
res = hypothetical_ans_model.generate_content(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""")

to_markdown(res.text)

> In the abyssal depths of the ocean, where sunlight fades into eternal darkness and pressure increases with each descending fathom, life adapts to the extreme with remarkable resilience.
> 
> At the core of this survival lies a unique set of adaptations that optimize energy conservation and maximize resource utilization. Organisms here harness the power of [unique chemical reactions] and [specialized organs] to extract sustenance from the meager organic matter that drifts down from the sunlit surface.
> 
> The frigid temperatures and high-pressure environment necessitate a [slowed metabolism] that dramatically reduces energy expenditure. Organisms have evolved [flexible structures] that withstand the crushing forces, and [specialized enzymes] that function optimally in the cold and darkness.
> 
> Many deep-sea creatures have developed [bioluminescent capabilities], creating their own light to attract mates or lure prey. These glowing displays illuminate the otherwise pitch-black surroundings, facilitating communication and survival.
> 
> Moreover, deep-sea life exhibits [symbiotic relationships] with other organisms. Some species form partnerships with [bacteria that provide nourishment], while others establish alliances with [larger predators that offer protection]. These codependencies foster survival in an environment where resources are scarce.
> 
> With their exceptional adaptations, deep-sea creatures defy the odds, thriving in an environment that would crush most other life forms. Their resilience and ingenuity serve as a testament to the boundless wonders that lie hidden in the unfathomable depths of our oceans.

使用 Gemini API 取得基礎答案並與搜尋結果進行比較


In [ ]:
hypothetical_ans = get_embeddings([res.text])

計算相似度得分以對搜尋結果進行排名


In [ ]:
sim_value = dot_product(search_res, hypothetical_ans)

In [ ]:
sim_value

array([[0.74144238],
       [0.81955475],
       [0.83515347],
       [0.79930045],
       [0.78609111],
       [0.83174707],
       [0.77690633],
       [0.69279512],
       [0.69031457],
       [0.73286281],
       [0.73241528],
       [0.74342376],
       [0.79973473],
       [0.67441962],
       [0.70701801]])

使用 `np.argmax` 即可選擇最佳候選項。

**使用者輸入：** 說明深海生物如何生存。

**回覆：** 


In [ ]:
to_markdown(summaries[np.argmax(sim_value)])

> Deep-sea creatures can endure the immense pressure in the deep sea due to a variety of adaptations. These adaptations include:
> 
> * **Having a body that is made up mostly of water.** This reduces the amount of pressure that the creature's body experiences.
> * **Having a thick, tough skin that is resistant to pressure.** This helps to protect the creature's body from the crushing pressure of the water.
> * **Having a skeleton that is made of a material that is stronger than bone.** This helps to support the creature's body and prevent it from collapsing under pressure.
> * **Having a circulatory system that is adapted to high pressure.** This helps to ensure that the creature can get the oxygen and nutrients it needs to survive.
> * **Having a metabolism that is slowed down.** This helps to conserve energy and prevent the creature from overheating.
> 
> These adaptations allow deep-sea creatures to survive in the extreme conditions of the deep sea.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Ocean

你現在已使用內嵌碼建立了搜尋重新排序引擎！


## 後續步驟

希望你發現這個範例有所幫助！檢閱 [Gemini 指南](https://github.com/google-gemini/gemini-guide/) 中更多範例以深入了解。
